In [ ]:
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM

dataset = load_dataset("json", data_files=["/content/our_mega_over_true_dataset.jsonl", "/ds.jsonl.xz"], split="train")

#--------[prompt_formatter.py]-------

ROLE_TOKEN = {
    'system': '[SYS]',
    'user':   '[USR]',
    'web':    '[WEB]',
    'bot':    '[BOT]'
}
S_END = '[/]'
S_PAD = '[_]'

def bulk_fmt(ds):
    O = []
    for chain in ds['messages']:
        c = ''
        for msg in chain:
            c += ROLE_TOKEN[msg['role']]
            c += msg['content']
            c += S_END
        O.append(c)
    return O

#-------------------------------------

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

# print(tokenizer('[USR]Example Prompt[/][BOT]Some Answer[/]'))


tokenizer.add_special_tokens({
    'eos_token':                 S_END,
    'pad_token':                 S_PAD,
    'additional_special_tokens': list(ROLE_TOKEN.values())
})

model.resize_token_embeddings(len(tokenizer))

# print(tokenizer('[USR]Example Prompt[/][BOT]Some Answer[/]'))

trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    max_seq_length=512,
    tokenizer=tokenizer,
    formatting_func=bulk_fmt,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        save_strategy = 'steps',
        save_steps = 1000,
        output_dir = 'drive/MyDrive/gpt-oasst'
    ),
)
trainer.train()


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/82815 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
500,4.472000
1000,3.222300
1500,3.171200
2000,3.176600
2500,3.073500
3000,3.070200
3500,2.962000
4000,2.987700
4500,3.048400
5000,2.966200


Checkpoint destination directory drive/MyDrive/gpt-oasst/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory drive/MyDrive/gpt-oasst/checkpoint-2000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory drive/MyDrive/gpt-oasst/checkpoint-3000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory drive/MyDrive/gpt-oasst/checkpoint-4000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory drive/MyDrive/gpt-oasst/checkpoint-5000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory drive/MyDrive/gpt-oasst/checkpoint-6000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory drive/M

KeyboardInterrupt: 

In [ ]:
from google.colab import files
files.download("/content/drive/MyDrive/gpt-oasst/checkpoint-79000")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r checkpoint-79000.zip /content/drive/MyDrive/gpt-oasst/checkpoint-79000

  adding: content/drive/MyDrive/gpt-oasst/checkpoint-79000/ (stored 0%)
  adding: content/drive/MyDrive/gpt-oasst/checkpoint-79000/generation_config.json (deflated 24%)
  adding: content/drive/MyDrive/gpt-oasst/checkpoint-79000/model.safetensors (deflated 7%)
  adding: content/drive/MyDrive/gpt-oasst/checkpoint-79000/merges.txt (deflated 53%)
  adding: content/drive/MyDrive/gpt-oasst/checkpoint-79000/tokenizer.json (deflated 72%)
  adding: content/drive/MyDrive/gpt-oasst/checkpoint-79000/vocab.json (deflated 59%)
  adding: content/drive/MyDrive/gpt-oasst/checkpoint-79000/training_args.bin (deflated 51%)
  adding: content/drive/MyDrive/gpt-oasst/checkpoint-79000/scheduler.pt (deflated 55%)
  adding: content/drive/MyDrive/gpt-oasst/checkpoint-79000/trainer_state.json (deflated 80%)
  adding: content/drive/MyDrive/gpt-oasst/checkpoint-79000/config.json (deflated 51%)
  adding: content/drive/MyDrive/gpt-oasst/checkpoint-79000/tokenizer_config.json (deflated 79%)
  adding: content/drive/MyD

In [ ]:
!pwd

/content


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/gpt-oasst/checkpoint-79000")

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/gpt-oasst/checkpoint-79000", device="cuda:0")


input_text = "[USR]What time is it now?[/][WEB]now 10:22[/][BOT]"
input_ids = tokenizer(input_text, return_tensors="pt")


outputs = model.generate(**input_ids, max_length = 512, repetition_penalty = 10., encoder_repetition_penalty = 10., eos_token_id = tokenizer(S_END)['input_ids'][0])
yt = tokenizer.decode(outputs[0])

print(yt[(yt.index('[BOT]') + 5):-3])
# print(input_ids)

Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.


[USR]What time is it now?[/][WEB]now 10:22[/][BOT]I'm sorry, but the time is not right now.[/]


In [ ]:
tokenizer(S_END)['input_ids'][0]

[50257]

In [ ]:
!xz --decompress ds.jsonl.xz

xz: ds.jsonl.xz: No such file or directory


In [ ]:
!pip install accelerate
!pip install datasets
!pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━